In [ ]:
# change to the root directory of the project
import os
if os.getcwd().split("/")[-1] == "examples":
    os.chdir('..')
print(os.getcwd())

If we want to run this notebook on Google Colab, we first have to install `ScanDy` and download the required dataset from Google drive. The following code cell will prepare all of this for us.

In [ ]:
# install the ScanDy framework via pip
!pip install scandy

# download the VidCom_example dataset from google drive using gdown
!pip install gdown
# dataset is stored at https://drive.google.com/file/d/1oT9OJ2tRsvdJGFFLSKDCaY3BJev4Irzf/view?usp=sharing
file_id = '1oT9OJ2tRsvdJGFFLSKDCaY3BJev4Irzf'
url = f"https://drive.google.com/uc?id={file_id}"
output = 'vidcom_example.zip'
!gdown $url -O $output
!unzip $output

# make visualizations directory for the output (if not cloned from github)
!mkdir visualizations

In [ ]:
import numpy as np
import pandas as pd
import random
import seaborn as sns
import matplotlib.pyplot as plt

from scandy.models.LocationModel import LocationModel
from scandy.models.ObjectModel import ObjectModel
from scandy.utils.dataclass import Dataset
import scandy.utils.functions as uf

from neurolib.utils.parameterSpace import ParameterSpace
from neurolib.optimize.evolution import Evolution

from IPython import display

# Visualization of simulated scanpaths

The visualizations created in this notebook are stored in the `ScanDy/visualizations/` folder.

## Load dataset
ScanDy assumes video information to be already precomputed. The paths for the precomputed maps can be provided when initializing the dataset. If no information is given, it assumes the following file structure:

```
DATAPATH/
├── videos/                 # Folder containing the videos (only for visualization)
├── featuremaps/            # Folder containing the precomputed saliency maps
    ├── molin/              #   The name of the subfolder is not required,   
    ├── TASEDnet/           #   but has to match params['featureset'].
    └── /.../ 
├── polished_segmentation/  # Folder with object segmentation masks
├── optical_flow/           # Folder with optical flow maps (e.g. PWC net)
└── gt_fov_maps_333/        # Optional, if NSS scores are to be computed 
                            # (smoothed human gaze positions)
```

In [ ]:
datadict = {
    "PATH": "VidCom_example/",  # previously downloaded & extracted dataset  
    'FPS' : 30,
    'PX_TO_DVA' : 0.06,
    'FRAMES_ALL_VIDS' : 300,
    'gt_foveation_df' : 'VidCom_GT_fov_df.csv',
    "outputpath" : os.getcwd()+"/visualizations/"  # path for saving the visualizations
}
VidCom = Dataset(datadict)

## Initialize a model and specify parameters

We initialize an instance from the object-based model family. We here use low-level saliency maps from Molin et al., which corresponds to the `O_ll` model in the paper.

In [ ]:
O_ll = ObjectModel(VidCom)
# low level features
O_ll.params["featuretype"] = "molin"

We initialize the free model parameters with the average parameters from the evolutionary optimization described in the manuscript.

In [ ]:
O_ll.params["ddm_thres"] = 1.873
O_ll.params["ddm_sig"] = 0.241
O_ll.params["att_dva"] = 13.72
O_ll.params["ior_decay"] = 198.9
O_ll.params["ior_inobj"] = 0.76

## Run and evaluate for a single video

Given the model and the dataset, we can now run the scanpath simulation. First we only run it a single time and choose a random seed for reproducibility.

Running this should only take a few seconds.

In [ ]:
O_ll.run('field03', seeds = [10])

Let's have a look at the events (i.e. saccadic decisions and resulting foveations) of the simulated scanpath.

In [ ]:
O_ll.evaluate_all_to_df()
O_ll.result_df

We can now qualitatively assess how reasonable this predicted scanpath is by plotting it on top of the "observed" video.

## Visualize the different modules for a single scanpath

The models have a method that visualizes what's going on in the different modules (I-V) of the model while the scanpath is simulated. The creation of the gif will take multiple minutes and a lot of RAM (typically more than a laptop has or colab provides, so you might want to skip this cell and look at the visualizations provided on [GitHub](https://github.com/rederoth/ScanDy/tree/main/visualizations)).

In [ ]:
O_ll.write_sgl_output_gif('field03_Oll_mean_sglrun', slowgif=True, dpi=100)

This is then saved as a gif (specified outputpath in `Dataset`) and can be displayed in the notebook with the following command:

In [ ]:
display.Image(VidCom.outputpath + 'field03_Oll_mean_sglrun_slow.gif')

This gif shows the simulated gaze position (green cross) on top of visualizations of the different modules of the model. The bottom left panel shows the object masks on top of the original video (shown with 10 fps instead of 30 fps).

(I) Precomputed low-level saliency map with anisotropic center bias. Low values are shown in dark, high values in bright colors. 

(II) Gaze dependent visual sensitivity map, Gaussian with a uniform spread across currently foveated objects. Black means not sensitive (0), white means fully sensitive (1).

(III) Visualization of the inhibition of return value of each object (attribute of the `ObjectFile` instance). White means no inhibition (0), black means fully inhibited (1).

(IV) Visualization of the decision variable of each object (attribute of the `ObjectFile` instance). The saturation of the object mask represents the amount of accumulated evidence (white corresponds to 0, dark blue/red/green/orange to the decision threshold $\theta$). When an object (including the background in blue) is fully saturated, it means that the decision variable has reached the threshold and a saccade is made to (or within) this object. After a saccade, all decision variables are reset to zero.

(V) The red circle indicates the next gaze position. The pixel values indicate for each object how likely each position within each object is as a saccade target (calculated from the features (I) and sensitivity (II), $F\times S$). If no saccade is made, the gaze point moves with the currently foveated object, resulting in either fixation or smooth pursuit behavior.

## Simulate and visualize multiple scanpaths
Due to the stochasticity of the scanpath generation, a single run is not sufficient to assess the quality of the model predictions. We therefore run the simulation multiple times and plot the scanpaths on top of the video.

In [ ]:
O_ll.run('field03', seeds = [s for s in range(1, 13)], overwrite_old=True)
O_ll.evaluate_all_to_df(overwrite_old=True)

In [ ]:
O_ll.video_output_gif('field03', 'field03_Oll_mean', slowgif=False, dpi=100)

In this visualization every color corresponds to a different simulated scanpath (i.e., a different random seed). The video is shown with 30 fps, as in the eye tracking data collection.

In [ ]:
display.Image(VidCom.outputpath + 'field03_Oll_mean.gif')

## Location-based model

Lastly, we repeat the above steps for the location-based model with low-level features, `L_ll`.

In [ ]:
L_ll = LocationModel(VidCom)

L_ll.params["featuretype"] = "molin"
L_ll.params["ddm_thres"] = 0.355
L_ll.params["ddm_sig"] = 0.013
L_ll.params["att_dva"] = 12.77
L_ll.params["ior_decay"] = 226.5
L_ll.params["ior_dva"] = 6.82

L_ll.run('field03', seeds = [10])
L_ll.evaluate_all_to_df()
L_ll.result_df

In [ ]:
L_ll.write_sgl_output_gif('field03_Lll_mean_sglrun', slowgif=True, dpi=100)

Analogously to the object-based model above, this gif shows the simulated gaze position (green cross) on top of visualizations of the different modules of the model. The bottom left panel shows the original video (shown with 10 fps instead of 30 fps).

(I) Precomputed low-level saliency map with anisotropic center bias. Low values are shown in dark, high values in bright colors. 

(II) Gaze dependent Gaussian visual sensitivity map. Black means not sensitive (0), white means fully sensitive (1).

(III) Inhibition of return map (value calculated for every pixel). White means no inhibition (0), black means fully inhibited (1)

(IV) Visualization of the decision variable of each pixel-location. The saturation of a pixel represents the amount of accumulated evidence (white corresponds to 0, dark red to the decision threshold $\theta$).

(V) The red circle indicates the next gaze position. The pixel values indicate the optical flow.

In [ ]:
display.Image(VidCom.outputpath + 'field03_Lll_mean_sglrun_slow.gif')

In [ ]:
L_ll.run('field03', seeds = [s for s in range(1, 13)], overwrite_old=True)
L_ll.evaluate_all_to_df(overwrite_old=True)
L_ll.video_output_gif('field03', 'field03_Lll_mean', slowgif=False, dpi=100)

In [ ]:
display.Image(VidCom.outputpath + 'field03_Lll_mean.gif')

By just comparing the resulting scanpaths on this one video ("field03" is part of the test set), we can see that the location-based model is not able to appropriately capture the way humans would explore the scene. The object-based model, on the other hand, leads to scanpaths which are hard to distinguish from human scanpaths.

_______________________________________________________________________________________________